<a href="https://colab.research.google.com/github/cric96/DL-exercise/blob/main/DL_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Autoencoders tutorial**
In today's tutorial you will learn how to use autoencoders to solve the following tasks:
- dimensionality reduction;
- image denoising;
- anomaly detection.

We will use [**TensorFlow**](https://ekababisong.org/gcp-ml-seminar/tensorflow/) framework and [**Keras**](https://keras.io/) open-source library to rapidly prototype deep neural networks.

# **Preliminary operations**
The following code downloads all the necessary material into the remote machine. At the end of the execution select the **File** tab to verify that everything has been correctly downloaded.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00266/seismic-bumps.arff

# **Useful modules import**
First of all, it is necessary to import useful modules used during the tutorial.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from scipy.io import arff
from sklearn.decomposition import PCA
from tensorflow import keras
from sklearn.preprocessing import OrdinalEncoder
from tensorflow.keras import layers
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# **Utility functions**
Execute the following code to define some utility functions used in the tutorial:
- **plot_2d_data** plots 2D labeled data;
- **plot_history** draws in a graph the loss trend over epochs on both training and validation sets. Moreover, if provided, it draws in the same graph also the trend of the given metric;
- **show_confusion_matrix** visualizes a 2D confusion matrix as a color-coded image.

In [ ]:
def plot_2d_data(data_2d,y,titles=None,figsize=(7,7)):
  _,axs=plt.subplots(1,len(data_2d),figsize=figsize)

  for i in range(len(data_2d)):
    if (titles!=None):
      axs[i].set_title(titles[i])
    scatter=axs[i].scatter(data_2d[i][:,0],data_2d[i][:,1],s=1,c=y[i],cmap=plt.cm.Paired)
    axs[i].legend(*scatter.legend_elements())

def plot_history(history,metric=None):
  fig, ax1 = plt.subplots(figsize=(10, 8))

  epoch_count=len(history.history['loss'])

  line1,=ax1.plot(range(1,epoch_count+1),history.history['loss'],label='train_loss',color='orange')
  ax1.plot(range(1,epoch_count+1),history.history['val_loss'],label='val_loss',color = line1.get_color(), linestyle = '--')
  ax1.set_xlim([1,epoch_count])
  ax1.set_ylim([0, max(max(history.history['loss']),max(history.history['val_loss']))])
  ax1.set_ylabel('loss',color = line1.get_color())
  ax1.tick_params(axis='y', labelcolor=line1.get_color())
  ax1.set_xlabel('Epochs')
  _=ax1.legend(loc='lower left')

  if (metric!=None):
    ax2 = ax1.twinx()
    line2,=ax2.plot(range(1,epoch_count+1),history.history[metric],label='train_'+metric)
    ax2.plot(range(1,epoch_count+1),history.history['val_'+metric],label='val_'+metric,color = line2.get_color(), linestyle = '--')
    ax2.set_ylim([0, max(max(history.history[metric]),max(history.history['val_'+metric]))])
    ax2.set_ylabel(metric,color=line2.get_color())
    ax2.tick_params(axis='y', labelcolor=line2.get_color())
    _=ax2.legend(loc='upper right')

def show_confusion_matrix(conf_matrix,class_names):
  fig, ax = plt.subplots(figsize=(6,6))
  img=ax.matshow(conf_matrix)
  fig.colorbar(img)
  tick_marks = np.arange(len(class_names))
  _=plt.xticks(tick_marks, class_names,rotation=45)
  _=plt.yticks(tick_marks, class_names)
  _=plt.ylabel('Real')
  _=plt.xlabel('Predicted')

### **Model definition**
The following function creates an undercomplete autoencoder given:
- the number of input features (*input_count*);
- the number of neurons for each hidden layer (*neuron_count_per_hidden_layer*);
- the dimension of the latent space (*encoded_dim*);
- the string identifier of the activation function of the hidden layers (*hidden_activation*);
- the string identifier of the activation function of the output layer (*output_activation*).

In Keras, a sequential is a stack of layers where each layer has exactly one input and one output. It can be created by passing a list of layers to the  constructor [**keras.Sequential**](https://keras.io/guides/sequential_model/).

[**Keras layers API**](https://keras.io/api/layers/) offers a wide range of built-in layers ready for use, including:
- [**Input**](https://keras.io/api/layers/core_layers/input/) - the input of the model. Note that, you can also omit the **Input** layer. In that case the model doesn't have any weights until the first call to a training/evaluation method (since it isn't yet built);
- [**Dense**](https://keras.io/api/layers/core_layers/dense/) - a fully-connected layer.

To combine encoder and decoder together forming the autoencoder, the [**Model**](https://keras.io/api/models/model/) class provided by Keras is used. Input and output layers are passed to the constructor, then it groups layers into an object with training and inference features.

<u>Note that, the **build_autoencoder** function returns the encoder and the decoder models as well as the whole autoencoder.</u>

In [ ]:
def build_autoencoder(input_count, neuron_count_per_hidden_layer, encoded_dim, hidden_activation, output_activation):
  #Encoder
  encoder = keras.Sequential(name = 'encoder')
  input_layer = layers.Input(shape = input_count, name = 'encoder_input');
  encoder.add(input_layer)
    
  for neuron_count in neuron_count_per_hidden_layer:
    hidden_layer = layers.Dense(neuron_count, activation = hidden_activation)
    encoder.add(hidden_layer)
      
  latent_layer = layers.Dense(encoded_dim, activation = hidden_activation)
  encoder.add(latent_layer)
    
  #Decoder
  decoder = keras.Sequential(name = 'decoder')
  decoder.add(layers.Input(shape = encoded_dim))
  
  for neuron_count in reversed(neuron_count_per_hidden_layer):
    hidden_layer = layers.Dense(neuron_count, activation = hidden_activation)
    decoder.add(hidden_layer)
      
  output_layer = layers.Dense(input_count, activation = output_activation)
  decoder.add(output_layer)
  
  autoencoder = keras.Model(encoder.input, decoder(encoder.output), name = 'autoencoder')
    
  return autoencoder,encoder,decoder

# **Anomaly detection**

---


Mining activity was and is always connected with the occurrence of dangers which are commonly called
mining hazards. A special case of such threat is a seismic hazard which frequently occurs in many
underground mines. Seismic hazard is the hardest detectable and predictable of natural hazards and in
this respect it is comparable to an earthquake. More and more advanced seismic and seismoacoustic
monitoring systems allow a better understanding rock mass processes and definition of seismic hazard
prediction methods. Accuracy of so far created methods is however far from perfect. Complexity of
seismic processes and big disproportion between the number of low-energy seismic events and the number
of high-energy phenomena (e.g. > 10^4J) causes the statistical techniques to be insufficient to predict
seismic hazard. Therefore, it is essential to search for new opportunities of better hazard prediction,
also using machine learning methods. In seismic hazard assessment data clustering techniques can be
applied (Lesniak A., Isakow Z.: Space-time clustering of seismic events and hazard assessment in the
Zabrze-Bielszowice coal mine, Poland. Int. Journal of Rock Mechanics and Mining Sciences, 46(5), 2009,
918-928), and for prediction of seismic tremors artificial neural networks are used (Kabiesz, J.: Effect
of the form of data on the quality of mine tremors hazard forecasting using neural networks.
Geotechnical and Geological Engineering, 24(5), 2005, 1131-1147). In the majority of applications, the
results obtained by mentioned methods are reported in the form of two states which are interpreted as
'hazardous' and 'non-hazardous'. Unbalanced distribution of positive ('hazardous state') and negative
('non-hazardous state') examples is a serious problem in seismic hazard prediction. Currently used
methods are still insufficient to achieve good sensitivity and specificity of predictions. In the paper
(Bukowska M.: The probability of rockburst occurrence in the Upper Silesian Coal Basin area dependent on
natural mining conditions. Journal of Mining Sciences, 42(6), 2006, 570-577) a number of factors having
an effect on seismic hazard occurrence was proposed, among other factors, the occurrence of tremors with
energy > 10^4J was listed. The task of seismic prediction can be defined in different ways, but the main
aim of all seismic hazard assessment methods is to predict (with given precision relating to time and
date) of increased seismic activity which can cause a rockburst. In the data set each row contains a
summary statement about seismic activity in the rock mass within one shift (8 hours). If decision
attribute has the value 1, then in the next shift any seismic bump with an energy higher than 10^4 J was
registered. That task of hazards prediction bases on the relationship between the energy of recorded
tremors and seismoacoustic activity with the possibility of rockburst occurrence. Hence, such hazard
prognosis is not connected with accurate rockburst prediction. Moreover, with the information about the
possibility of hazardous situation occurrence, an appropriate supervision service can reduce a risk of
rockburst (e.g. by distressing shooting) or withdraw workers from the threatened area. Good prediction
of increased seismic activity is therefore a matter of great practical importance. The presented data
set is characterized by unbalanced distribution of positive and negative examples. In the data set there
are only 170 positive examples representing class 1.

In [ ]:
data = arff.loadarff('seismic-bumps.arff')
dataframe = pd.DataFrame(data[0])
ordinal_labels = ["seismic", "seismoacoustic", "ghazard", "class"]
ordinal_encoding = OrdinalEncoder()
dataframe[ordinal_labels] = ordinal_encoding.fit_transform(dataframe[ordinal_labels])
dataframe[["shift_N", "shift_W"]] = pd.get_dummies(dataframe['shift'], prefix='shift')
dataframe = dataframe.drop(columns = ["shift"])

The variable *dataframe* is an instance of the pandas class [**DataFrame**](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html), a 2-dimensional labeled data structure with columns of potentially different types.

### **Visualization**
*row_count* randomly selected rows can be shown by executing the following code.

In [ ]:
row_count = 5

dataframe.sample(row_count)

### **Statistics**
The [**info**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html) method can be used to print a brief summary of a **DataFrame** including the index and the type of each column, the non-null values and the memory usage.

In [ ]:
dataframe.info()

To show the overall statistics of the dataset can be used the method [**describe**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html).

In [ ]:
dataframe.describe().transpose()

The method [**hist**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html) draws a histogram for each column in the **DataFrame**.

In [ ]:
dataframe.hist(bins = 50, figsize = (20, 15))
plt.show()

### **Split features from target values**
The following code separates the features from the target values (clean/fraudulent transactions).

In [ ]:
dataframe_x = dataframe.drop(['class'], axis = 1)
dataframe_y = dataframe['class']

The Numpy representation of a **DataFrame** can be obtained using the [**values**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.values.html) property.

In [ ]:
x = dataframe_x.values
y = dataframe_y.values

print('Feature shape: ',x.shape)
print('Target shape: ',y.shape)

In [ ]:
cleans = y == 0
frauds = y == 1

clean_x = x[cleans]
clean_y = y[cleans]

fraud_x = x[frauds]
fraud_y = y[frauds]

print('Clean feature shape: ', clean_x.shape)
print('Clean target shape: ', clean_y.shape)
print('Fraudulent feature shape: ', fraud_x.shape)
print('Fraudulent target shape: ', fraud_y.shape)

### **Split data into train, validation and test sets**
In order to avoid overfitting during training and to evaluate the generalization capabilites of the models, it is necessary to divide the data into three disjoined datasets: training, validation and test sets.

For this reason, the data are divided using the **train_test_split** function provided by Scikit-learn.

The *test_size* and *val_size* parameters represent the percentage (or the absolute number) of patterns to include in the test and validation sets, respectively. 

<u>Note that, the autoencoder will be trained using only clean transactions while the test set will contain both clean and fraudulent transactions.</u>

In [ ]:
test_size = 0.10
val_size = 0.20

train_x, test_x, train_y, test_y = train_test_split(clean_x, clean_y, test_size = test_size,random_state = 1,shuffle=True)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = val_size,random_state = 1,shuffle=True)

test_x = np.concatenate((test_x, fraud_x))
test_y = np.concatenate((test_y, fraud_y))

print('Train feature shape: ',train_x.shape)
print('Train target shape: ',train_y.shape)
print('Validation feature shape: ',val_x.shape)
print('Validation target shape: ',val_y.shape)
print('Test feature shape: ',test_x.shape)
print('Test target shape: ',test_y.shape)

### **Data normalization**
It is good practice to normalize features that use different scales and ranges.

Scikit-learn library provides the class [**StandardScaler**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to normalize features by removing the mean and scaling to unit variance.

In [ ]:
scaler = StandardScaler().fit(train_x)
train_x = scaler.transform(train_x)
val_x = scaler.transform(val_x)
test_x = scaler.transform(test_x)

In [ ]:
pca = PCA(3)
x_pca = pca.fit_transform(train_x)
x_pca = pd.DataFrame(x_pca)
x_pca.columns = ['PC1','PC2','PC3']

In [ ]:
norm = plt.Normalize()
elements = train_y
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')

ax.scatter(x_pca.PC1, x_pca.PC2, x_pca.PC3, c = elements, alpha = 0.8)
plt.title('Scatter plot')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

## **The autoencoder**
In this section an autoencoder is trained to detect anolies into credit card transactions.

### **Model creation**
The following code creates the autoencoder by calling the **build_autoencoder** function defined above.

In [ ]:
autoencoder, encoder, _ = build_autoencoder(input_count = train_x.shape[1], neuron_count_per_hidden_layer = [256, 256, 128, 128, 64, 64], encoded_dim = 4, hidden_activation = 'elu', output_activation = None)

### **Model visualization**
A string summary of the network can be printed by executing the following code.

In [ ]:
autoencoder.summary()

lternatively, a plot of the neural network graph can be visualized.

In [ ]:
keras.utils.plot_model(autoencoder, show_shapes = True, show_layer_names = False, expand_nested = True)

### **Model compilation**
The following code compiles the model.

In [ ]:
autoencoder.compile(loss = 'mse', optimizer = 'adam')

### **Training**
Now we are ready to train our model by calling the **fit** method.

In [ ]:
epoch_count = 1000
batch_size = 64
patience = 50

early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = patience, restore_best_weights = True)

history = autoencoder.fit(train_x, train_x, validation_data = (val_x, val_x), epochs = epoch_count, batch_size = batch_size, callbacks = [early_stop])

The following code calls the **plot_history** function defined above to draws in a graph the loss over epochs on both training and validation sets.

In [ ]:
plot_history(history)

## **Latent space visualization**
It is always interesting to look at the compressed representation obtained by the autoencoder.

The **predict** method of the *encoder* can be used to reduce training, validation and test sets.

In [ ]:
encoded_train_x = encoder.predict(train_x)
encoded_val_x = encoder.predict(val_x)
encoded_test_x = encoder.predict(test_x)

The following code visualize training,validation and test sets mapped into the latent space.

In [ ]:
plot_2d_data([encoded_train_x, encoded_val_x, encoded_test_x],[train_y, val_y, test_y],['Train','Validation','Test'],(15,7))

## **Anomaly detection**


In [ ]:
reconstructed_train_x = autoencoder.predict(train_x)
reconstructed_val_x = autoencoder.predict(val_x)
reconstructed_test_x=autoencoder.predict(test_x)

print('Reconstructed train shape: ',reconstructed_train_x.shape)
print('Reconstructed validation shape: ',reconstructed_val_x.shape)
print('Reconstructed test shape: ',reconstructed_test_x.shape)

Scikit-learn library provides the function [**mean_squared_error**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) to compute MSE metric.

In [ ]:
train_mse=mean_squared_error(train_x.transpose(),reconstructed_train_x.transpose(),multioutput='raw_values')
val_mse=mean_squared_error(val_x.transpose(),reconstructed_val_x.transpose(),multioutput='raw_values')
test_mse=mean_squared_error(test_x.transpose(),reconstructed_test_x.transpose(),multioutput='raw_values')

print('Train MSE shape: ',train_mse.shape)
print('Validation MSE shape: ',val_mse.shape)
print('Test MSE shape: ',test_mse.shape)

### **Distribution of means squared error**
The following code draws the MSE distributions of training, validation and test sets.

In [ ]:
_, axs = plt.subplots(1,3,figsize=(15,5))

axs[0].hist(train_mse, bins=100, density=True, label="clean", alpha=.6, color="green")
axs[0].set_title('Train')

axs[1].hist(val_mse, bins=100, density=True, label="clean", alpha=.6, color="green")
axs[1].set_title('Validation')

axs[2].hist(test_mse[(test_y==0).squeeze()], bins=100, density=True, label="clean", alpha=.6, color="green")
axs[2].hist(test_mse[(test_y==1).squeeze()], bins=100, density=True, label="fraudulent", alpha=.6, color="red")
axs[2].set_title('Test')

plt.legend()
plt.show()

Looking at the test distribution, although some fraudulent transactions present a low MSE very similar to clean transactions, in general the fraudulent transactions clearly have a distinguishing element in their data that sets them apart from clean ones.

### **Detection accuracy**
To detect fraudulent transactions a threshold on the MSE value can be used. 

It must be chosen to limit as much as possible the amount of clean transactions classified as fraudulent (i.e., false positive) and to capture the most anomalous ones.

Here we select as threshold the MSE value to obtain a specific percentage of true negatives on the validation set. 

The MSE value corresponding to a specific percentage (*clean_acceptance_rate*) of true negatives (i.e., clean transactions correctly classified) on the validation set is chosen as threshold.

In [ ]:
clean_acceptance_rate = 0.90

sorted_val_mse=np.sort(val_mse)

idx=int(clean_acceptance_rate*len(sorted_val_mse))

thr=sorted_val_mse[int(clean_acceptance_rate*len(sorted_val_mse))]

print('Anomaly detection threshold: {:.3f}'.format(thr))

The accuracy can be easily measured by calling the [**accuracy_score**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) method provided by the Scikit-learn library.

In [ ]:
train_y_pred=train_mse > thr
val_y_pred=val_mse > thr
test_y_pred=test_mse > thr

train_accuracy = accuracy_score(train_y,train_y_pred,normalize='true')
val_accuracy = accuracy_score(val_y,val_y_pred,normalize='true')
test_accuracy = accuracy_score(test_y,test_y_pred,normalize='true')

print('Train accuracy: {:.3f}'.format(train_accuracy))
print('Validation accuracy: {:.3f}'.format(val_accuracy))
print('Test accuracy: {:.3f}'.format(test_accuracy))

### **Confusion matrix**
To evaluate the classification accuracy in presence of an unbalanced dataset, it is useful to compute the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix).

Scikit-learn library provides the function [**confusion_matrix**](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) to compute the confusion matrix given the grouhd truth (*test_y*) and the predicted classes (*test_y_pred*) as input.

In [ ]:
conf_matrix=confusion_matrix(test_y, test_y_pred, normalize='true')
print(conf_matrix)

The following code visualizes the 2D confusion matrix as a color-coded image.

In [ ]:
show_confusion_matrix(conf_matrix,('normal','problem'))